In [29]:
import pickle
import pandas as pd 

map_filename = './data_parl_map.pickle'
district_era_filename = './district_eras_map.pickle'


In [30]:
class RidingObject:
    def __init__(self, name):
        self.name = name
        self.eras = []
        self.elections = []

class Era:
    def __init__(self, start, end, predecessors, successors):
        self.start = start
        self.end = end
        self.predecessors = predecessors
        self.successors = successors

    def add_dates(self, start, end):
        self.start = start
        self.end = end

    def add_predecessors (self, predecessors):
        self.predecessors = predecessors

    def add_successors(self, successors):
        self.successors = successors

In [31]:
with open(map_filename, 'rb') as handle:
    pickle_map = pickle.load(handle)
print(list(pickle_map.items()))

[('Bonavista—Twillingate', 'Bonavista--Twillingate'), ('Burin—Burgeo', 'Burin--Burgeo'), ('Grand Falls—White Bay', 'Grand Falls--White Bay'), ("Humber—St. George's", "Humber--St. George's"), ("St. John's East", "St. John's East"), ("St. John's West", "St. John's West"), ('Trinity—Conception', 'Trinity--Conception'), ('Annapolis—Kings', 'Annapolis--Kings'), ('Antigonish—Guysborough', 'Antigonish--Guysborough'), ('Cape Breton North and Victoria', 'Cape Breton North and Victoria'), ('Cape Breton South (federal electoral district)', 'Cape Breton South'), ('Colchester—Hants', 'Colchester--Hants'), ('Cumberland (electoral district)', 'Cumberland'), ('Digby—Yarmouth', 'Digby--Yarmouth'), ('Halifax (electoral district)', 'Halifax'), ('Inverness—Richmond', 'Inverness--Richmond'), ('Lunenburg (electoral district)', 'Lunenburg'), ('Pictou (electoral district)', 'Pictou'), ('Queens—Shelburne', 'Queens--Shelburne'), ("King's (Prince Edward Island electoral district)", "King's"), ('Prince (electoral

In [32]:
with open(district_era_filename, 'rb') as handle:
    scraped_data_map = pickle.load(handle)
print(list(scraped_data_map.items()))

[('Vancouver Centre', <__main__.RidingObject object at 0x7f7486a8a250>)]


In [33]:
filename = "./federal_election_data.csv"

elec_data = pd.read_csv(filename, sep=',', delimiter=None, header='infer')
elec_data['score'] = np.zeros(len(elec_data))
print(elec_data.columns)
#print(pickle_map['vancouver centre'])
 
relevant_data = elec_data.loc[elec_data['riding'] == 'vancouver centre']
print(relevant_data)


Index(['riding', 'electiondate', 'provincecode', 'lastname', 'firstname',
       'parliament', 'byelec', 'other', 'lib', 'ndp', 'green', 'bloc',
       'allcons', 'won', 'score'],
      dtype='object')
                riding electiondate  provincecode    lastname  \
3814  vancouver centre   1921-12-06             2      BATSON   
3815  vancouver centre   1921-12-06             2     STEVENS   
3816  vancouver centre   1921-12-06             2        GALE   
3817  vancouver centre   1921-12-06             2    O'CONNOR   
3850  vancouver centre   1925-10-29             2     LEFEAUX   
...                ...          ...           ...         ...   
7174  vancouver centre   2008-10-14             2        CARR   
7175  vancouver centre   2008-10-14             2         FRY   
7176  vancouver centre   2008-10-14             2  MAYENCOURT   
7177  vancouver centre   2008-10-14             2       BYERS   
7178  vancouver centre   2008-10-14             2      CLARKE   

             firs

In [34]:
election_dates = relevant_data['electiondate']
print(len(election_dates))
print(set(election_dates))


169
{'1921-12-06', '2008-10-14', '1949-06-27', '1945-06-11', '1962-06-18', '1980-02-18', '1940-03-26', '1979-05-22', '1948-06-08', '1972-10-30', '1958-03-31', '1988-11-21', '1963-04-08', '1926-09-14', '1997-06-02', '1974-07-08', '2004-06-28', '1965-11-08', '1957-06-10', '1930-07-28', '1968-06-25', '1935-10-14', '1993-10-25', '2000-11-27', '1953-08-10', '1925-10-29', '1984-09-04', '2006-01-23'}


In [35]:
import datetime
 
election_dates = list(set(election_dates))
 
election_dates = [datetime.datetime.strptime(date, "%Y-%m-%d") for date in election_dates]
election_dates.sort()
sorted_election_dates = [datetime.datetime.strftime(date, "%Y-%m-%d") for date in election_dates]

print(sorted_election_dates)


['1921-12-06', '1925-10-29', '1926-09-14', '1930-07-28', '1935-10-14', '1940-03-26', '1945-06-11', '1948-06-08', '1949-06-27', '1953-08-10', '1957-06-10', '1958-03-31', '1962-06-18', '1963-04-08', '1965-11-08', '1968-06-25', '1972-10-30', '1974-07-08', '1979-05-22', '1980-02-18', '1984-09-04', '1988-11-21', '1993-10-25', '1997-06-02', '2000-11-27', '2004-06-28', '2006-01-23', '2008-10-14']


In [36]:
import numpy as np
 
election_competition = np.zeros_like(election_dates)
winner_scores = np.zeros_like(election_dates)

def compute_score(index, party):
    score = 0
    if (index > 0 and election_competition[index - 1] == party):
        score += 3
    if (index -1 > 0 and election_competition[index - 2] == party):
        score += 2
    if (index -2 > 0 and election_competition[index - 3] == party):
        score += 1
    return score
compute_score_vec = np.vectorize(compute_score)
 
for idx, election_date in enumerate(sorted_election_dates):
    election_data = relevant_data.loc[relevant_data['electiondate'] == election_date]
    #print('############################################################')
    #print(election_data)
    candidates_data = np.array(election_data.iloc[:, 7:-2])
    candidate_parties = np.argmax(candidates_data, axis=1)
    results_data = np.array(election_data.iloc[:, -1:])

    winner = np.argmax(results_data)
    winner_party = candidate_parties[winner]
    election_competition[idx] = winner_party

    candidate_scores = compute_score_vec(idx, candidate_parties)
    relevant_data.loc[relevant_data['electiondate'] == election_date, 'score'] = candidate_scores
    #print(relevant_data[relevant_data['electiondate'] == election_date])

    if idx > 4:
        break

In [37]:
"""
0 - other 
1 - lib
2 - ndp
3 - green
4 - bloc
5 - cons
"""


def get_score(index):
    if (index < 0):
        return -1
    else:
        return election_competition[idx]
 
print(election_competition)

# For each index
# Get score for previous elections

[0 0 1 5 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [38]:
print(relevant_data)

riding electiondate  provincecode    lastname  \
3814  vancouver centre   1921-12-06             2      BATSON   
3815  vancouver centre   1921-12-06             2     STEVENS   
3816  vancouver centre   1921-12-06             2        GALE   
3817  vancouver centre   1921-12-06             2    O'CONNOR   
3850  vancouver centre   1925-10-29             2     LEFEAUX   
...                ...          ...           ...         ...   
7174  vancouver centre   2008-10-14             2        CARR   
7175  vancouver centre   2008-10-14             2         FRY   
7176  vancouver centre   2008-10-14             2  MAYENCOURT   
7177  vancouver centre   2008-10-14             2       BYERS   
7178  vancouver centre   2008-10-14             2      CLARKE   

             firstname  parliament  byelec  other  lib  ndp  green  bloc  \
3814  CADWALLADERFLAGG          14       0      1    0    0      0     0   
3815      HENRYHERBERT          14       0      0    0    0      0     0   
3816   

In [39]:
# COMPETITIVENESS COMPUTATION
"""
-Need data on the previous 3 elections

At each election points are awarded for having won preceding elections in that riding

Election | points
-------------------
N - 1 |   3
-------------------
N - 2 |   2
-------------------
N - 3 |   1

"""

'\n-Need data on the previous 3 elections\n\nAt each election points are awarded for having won preceding elections in that riding\n\nElection | points\n-------------------\nN - 1 |   3\n-------------------\nN - 2 |   2\n-------------------\nN - 3 |   1\n\n'

In [40]:
# must be done chronologically
# for each election date in order

In [41]:
# Get example of recent riding

In [42]:
# For election in riding
    # Obtain relevant preceding election data
    # For candidate in election
    # 